In [1]:
import h5py
import numpy as np
import os
from queue import Queue
import random

## play style public

In [2]:
df = open('./CSVs/play_style_test_public.csv').read().splitlines()
games = [i.split(',',1)[1] for i in df]

## play style private

In [3]:
# df = open('./CSVs/play_style_test_private_f.csv').read().splitlines()
# games = [i.split(',',1)[1] for i in df]

In [3]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
chars_reverse = 'srqponmlkjihgfedcba'
coordinates_reverse = {k:v for v,k in enumerate(chars_reverse)}

coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

## Feature Map(共28個)

    0:player board
    1:opponent board
    2~5:player的氣數 (1, 2, 3, >=4)
    6~9:opponent的氣數(1, 2, 3, >=4)
    10:標示空地
    11~18:最後八步
    19~21:最後一步的周圍7*7(player, opponent, 空位)
    22~24:最後一步的周圍5*5(player, opponent, 空位)
    25~27:最後一步的周圍3*3(player, opponent, 空位)

In [4]:
player_board = 0
oppnent_board = 1
player_air_1 = 2
player_air_2 = 3
player_air_3 = 4
player_air_4 = 5
oppnent_air_1 = 6
oppnent_air_2 = 7
oppnent_air_3 = 8
oppnent_air_4 = 9
empty_board = 10
last_1 = 11
last_8 = 18
round_7 = 19
round_5 = 22
round_3 = 25

In [5]:
liber = Queue()
def count_air2(move):
    bfs = []
    col = coordinates[move[2]]
    row = coordinates[move[3]]
    #找四周要bfs的
    if col - 1 >= 0 and x[row][col - 1][empty_board] == 0:
        bfs.append([row, col - 1])
    if col + 1 < 19 and x[row][col + 1][empty_board] == 0:
        bfs.append([row, col + 1])
    if row - 1 >= 0 and x[row - 1][col][empty_board] == 0:
        bfs.append([row - 1, col])
    if row + 1 < 19 and x[row + 1][col][empty_board] == 0:
        bfs.append([row + 1, col])
    bfs.append([row, col])
    for m in bfs:
        BFS(m)
    bfs = []
    if not liber.empty():
        while not liber.empty():
            li_row, li_col = liber.get()
            if li_col - 1 >= 0 and x[li_row][li_col - 1][empty_board] == 0:
                bfs.append([li_row, li_col - 1])
            if li_col + 1 < 19 and x[li_row][li_col + 1][empty_board] == 0:
                bfs.append([li_row, li_col + 1])
            if li_row - 1 >= 0 and x[li_row - 1][li_col][empty_board] == 0:
                bfs.append([li_row - 1, li_col])
            if li_row + 1 < 19 and x[li_row + 1][li_col][empty_board] == 0:
                bfs.append([li_row + 1, li_col])
        for m in bfs:
            BFS(m)
        
def BFS(now):
    q = Queue()
    row = now[0]
    col = now[1]
    q.put((row, col))
    visited = [[False for _ in range(19)] for _ in range(19)]
    nodes = []
    air = 0
    next_board = player_air_1
    board = player
    if oppnent[row, col] == 1:
        board = oppnent
        next_board = oppnent_air_1
    while not q.empty():
        m, n = q.get()
        if m < 0 or n < 0 or m >= 19 or n >= 19 or visited[m][n]:
            continue
        visited[m][n] = True
        if board[m, n] == 1:
            nodes.append([m, n])
            q.put((m - 1, n))
            q.put((m + 1, n))
            q.put((m, n - 1))
            q.put((m, n + 1))
        elif x[m, n, empty_board] == 1: #空地 -> 自由度+1
            air += 1
    dele = -1
    for i in range(player_air_1, player_air_1 + 8):
        if x[row][col][i] == 1:
            dele = i
            break
    air = min(air, 4)
    for node in nodes:
        n_row = node[0]
        n_col = node[1]
        for i in range(player_air_1, player_air_1 + 8):
            x[n_row][n_col][i] = 0;
        if air == 0:
            x[n_row][n_col][empty_board] = 1
            player[n_row, n_col] = 0
            oppnent[n_row, n_col] = 0
            liber.put(node)
        else:
            x[n_row][n_col][next_board + air - 1] = 1

In [6]:
x = np.zeros((19,19,28))
player = np.zeros((19, 19))
oppnent = np.zeros((19, 19))


def prepare_input(moves, player_color):
    sz = len(moves)
    move = moves[-1]
    
    color = move[0]
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    if color == player_color: 
        player[row,column] = 1
    else: 
        oppnent[row,column] = 1
    x[row,column,empty_board] = 0
    
    #倒數8步
    for i in range(last_8, last_1, -1):
        x[:, :, i] = x[:, :, i - 1]
    x[:, :, last_1] = 0
    x[row, column, last_1] = 1
    
    count_air2(move)
    #周圍
    x[:, :, round_7:round_3 + 3] = 0
    last_col = coordinates[move[2]]
    last_row = coordinates[move[3]]
    for rad in range(3, 0, -1):
        row1 = max(0, last_row - rad)
        row7 = min(18, last_row + rad)
        col1 = max(0, last_col - rad)
        col7 = min(18, last_col + rad)
        for i in range(row1, row7 + 1, 1):
            for j in range(col1, col7 + 1, 1):
                x[i, j, round_7 + (3 - rad) * 3] = player[i, j]
                x[i, j, round_7 + 1 + (3 - rad) * 3] = oppnent[i, j]
                x[i, j, round_7 + 2 + (3 - rad) * 3] = x[i, j, empty_board]
                
    x[:, :, player_board] = player.copy()
    x[:, :, oppnent_board] = oppnent.copy()
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

def prepare_label_rotate90(move):
    column = coordinates[move[2]]
    row = coordinates_reverse[move[3]]
    return row*19+column

In [7]:
from IPython.display import clear_output

if os.path.isfile('./hdf5/test.hdf5'):
    os.remove('./hdf5/test.hdf5')
hdf5 = h5py.File('./hdf5/test.hdf5', 'a')
hdf5.create_dataset('Xs', (0, 19, 19, 28), maxshape=(None, 19, 19, 28), chunks=(10, 19, 19, 28)
                    ,dtype=np.float32, compression="lzf")
x_dataset = hdf5['Xs']

x_batch = []
batch_size = 1
print(f'total{len(games)}')
for game_i, game in enumerate(games): # Iterate through games
        clear_output(wait=True)
        print(game_i, end = ' ')
        x = np.zeros((19,19,28))
        player = np.zeros((19, 19))
        oppnent = np.zeros((19, 19))
        x[:, :, empty_board] = np.ones((19, 19))
        moves_list = game.split(',')
        for count, move in enumerate(moves_list):
            pi = prepare_input(moves_list[:count + 1], moves_list[-1][0])
            if count == len(moves_list) - 1:
                x_batch.append(pi.copy())
                if len(x_batch) >= batch_size or (game_i == len(games) - 1 
                                        and (count == len(moves_list) - 1 or count == len(moves_list) - 2)): # Yield when reached batch size
                        x_dataset.resize(x_dataset.shape[0] + len(x_batch), axis = 0)
                        x_dataset[-len(x_batch):] = np.array(x_batch)
                        x_batch = []
hdf5.close()

128 

KeyboardInterrupt: 